In [1]:
import tensorflow as tf
import scipy as sp
import numpy as np
import pandas as pd
import re
import hashlib
from sklearn.preprocessing import StandardScaler

import os
from pathlib import Path
import IPython.display as ipd
import cProfile

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
%matplotlib inline

from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.callbacks import ModelCheckpoint

from tensorflow.python.keras.utils import to_categorical

from sklearn.preprocessing import OrdinalEncoder

from tensorflow.python.keras.models import Sequential

from tensorflow.python.keras.layers import Input, Dense, LSTM

from tensorflow.python.keras.layers import Embedding, Dropout, Activation
from tensorflow.python.keras.layers import BatchNormalization
import sys
sys.path.insert(0, '../src/data')
import get_train_test
import keras
import keras.backend as K
from itertools import product
import cnn_utils


import math


Using TensorFlow backend.


In [22]:
from SGDRScheduler import SGDRScheduler

In [23]:
#keras gpu options
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
#config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

In [24]:
raw_train, raw_dev = get_train_test.get_train_test('../data/raw/train/audio/',10,3)

1000 : 2.9293775830788036 s
2000 : 4.5098847260592265 s
3000 : 6.1976108288085925 s
4000 : 7.6962573918008275 s


KeyboardInterrupt: 

In [ ]:
x_train = np.array([np.pad(x[0], ((0,99 - x[0].shape[0]),(0,0)), 'constant', constant_values=(0)) for x in raw_train])
x_dev = np.array([np.pad(x[0], ((0,99 - x[0].shape[0]),(0,0)), 'constant', constant_values=(0)) for x in raw_dev])

y_train = np.array([x[1] for x in raw_train])
y_dev = np.array([x[1] for x in raw_dev])

In [ ]:
# unk_train = np.where(y_train == 'unknown')[0]
unk_dev = np.where(y_dev == 'unknown')[0]

np.random.seed(42)
# unk_train_downsampled = np.random.choice(unk_train, size=unk_train.shape[0] - int(x_train.shape[0]/27), replace=False)
unk_dev_downsampled = np.random.choice(unk_dev, size=unk_dev.shape[0] - int(x_dev.shape[0]/27), replace=False)

# all_train_index = np.where(y_train)[0]
all_dev_index = np.where(y_dev)[0]

# downsampled_train_index = np.setdiff1d(all_train_index,unk_train_downsampled)
downsampled_dev_index = np.setdiff1d(all_dev_index,unk_dev_downsampled)

# x_train = x_train[downsampled_train_index]
x_dev_down = x_dev[downsampled_dev_index]

# y_train = y_train[downsampled_train_index]
y_dev_down = y_dev[downsampled_dev_index]



In [ ]:
x_train = x_train.reshape(-1,99,161,1)
x_dev_down = x_dev_down.reshape(-1,99,161,1)

In [ ]:
class_weight = {0: 37,
                1: 37,
                2: 37,
                3: 37,
                4: 37,
                5: 37,
                6: 37,
                7: 37,
                8: 2,
                9: 37,
                10: 37}

In [ ]:
x_resh = x_train.reshape(-1, 99*161)
scaler = StandardScaler()

scaler.fit(x_resh)

x_train_scaled = scaler.transform(x_resh).reshape(-1,99,161,1)
x_dev_scaled = scaler.transform(x_dev_down.reshape(-1,99*161)).reshape(-1,99,161,1)



In [ ]:
y_train_cat = to_categorical(OrdinalEncoder().fit_transform(y_train.reshape(-1,1)))
y_dev_cat = to_categorical(OrdinalEncoder().fit_transform(y_dev_down.reshape(-1,1)))

In [2]:
#LSTM Model
np.random.seed(42)

model =  cnn_utils.get_lstm_model()
checkpointer = ModelCheckpoint(filepath='./model-{val_categorical_accuracy:.3f}.hdf5', monitor='val_categorical_accuracy', verbose=1, save_best_only=True)

epoch_size = x_train_scaled.shape[0]
batch_size = 1024
schedule = SGDRScheduler(min_lr=1e-5,
                                     max_lr=3e-3,
                                     steps_per_epoch=np.ceil(epoch_size/batch_size),
                                     lr_decay=0.9,
                                     cycle_length=5,
                                     mult_factor=1.3)
np.random.seed(42)

model.fit(x_train_scaled.reshape(-1,99,161),y_train_cat, batch_size = batch_size, epochs = 50, shuffle = True,class_weight=class_weight,
                        validation_data=(x_dev_scaled.reshape(-1,99,161),y_dev_cat), callbacks=[checkpointer,schedule])

NameError: name 'LSTM' is not defined

In [13]:
#CNN Model


np.random.seed(42)
aml_model = cnn_utils.get_aml_model(x_train_scaled)

checkpointer = ModelCheckpoint(filepath='./model-{val_acc:.3f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True)
epoch_size = x_train_scaled.shape[0]
batch_size = 64
schedule = SGDRScheduler(min_lr=1e-5,
                                     max_lr=3e-3,
                                     steps_per_epoch=np.ceil(epoch_size/batch_size),
                                     lr_decay=0.9,
                                     cycle_length=5,
                                     mult_factor=1.3)
np.random.seed(42)
aml_model.fit(x_train_scaled,y_train_cat, batch_size = batch_size, epochs = 50, shuffle = True,class_weight=class_weight,
                        validation_data=(x_dev_scaled,y_dev_cat), callbacks=[ checkpointer])

Train on 19485 samples, validate on 840 samples
Epoch 1/50
19485/19485 [==============================] - 114s 6ms/step - loss: 148.9912 - acc: 0.1362 - val_loss: 10.1665 - val_acc: 0.1726

Epoch 00001: val_acc improved from -inf to 0.17262, saving model to ./model-0.173.hdf5
Epoch 2/50
19485/19485 [==============================] - 114s 6ms/step - loss: 43.3314 - acc: 0.3213 - val_loss: 2.5189 - val_acc: 0.2821

Epoch 00002: val_acc improved from 0.17262 to 0.28214, saving model to ./model-0.282.hdf5
Epoch 3/50
19485/19485 [==============================] - 114s 6ms/step - loss: 22.8882 - acc: 0.4977 - val_loss: 1.0666 - val_acc: 0.6738

Epoch 00003: val_acc improved from 0.28214 to 0.67381, saving model to ./model-0.674.hdf5
Epoch 4/50
19485/19485 [==============================] - 114s 6ms/step - loss: 18.4699 - acc: 0.5840 - val_loss: 0.7661 - val_acc: 0.8024

Epoch 00004: val_acc improved from 0.67381 to 0.80238, saving model to ./model-0.802.hdf5
Epoch 5/50
19485/19485 [=========

KeyboardInterrupt: 

In [14]:
from keras import backend as K

In [15]:
print(K.get_value(aml_model.optimizer.lr))

0.001


In [16]:
K.set_value(aml_model.optimizer.lr, 0.0004)

In [17]:
aml_model.fit(x_train_scaled,y_train_cat, batch_size = batch_size, epochs = 50, shuffle = True,class_weight=class_weight,
                        validation_data=(x_dev_scaled,y_dev_cat), callbacks=[ checkpointer])

Train on 19485 samples, validate on 840 samples
Epoch 1/50
19485/19485 [==============================] - 97s 5ms/step - loss: 3.4302 - acc: 0.8345 - val_loss: 0.4881 - val_acc: 0.9131

Epoch 00001: val_acc improved from 0.88690 to 0.91310, saving model to ./model-0.913.hdf5
Epoch 2/50
19485/19485 [==============================] - 99s 5ms/step - loss: 2.7595 - acc: 0.8508 - val_loss: 0.4866 - val_acc: 0.9155

Epoch 00002: val_acc improved from 0.91310 to 0.91548, saving model to ./model-0.915.hdf5
Epoch 3/50
19485/19485 [==============================] - 101s 5ms/step - loss: 2.4574 - acc: 0.8655 - val_loss: 0.4525 - val_acc: 0.9238

Epoch 00003: val_acc improved from 0.91548 to 0.92381, saving model to ./model-0.924.hdf5
Epoch 4/50
19485/19485 [==============================] - 100s 5ms/step - loss: 2.0780 - acc: 0.8783 - val_loss: 0.4883 - val_acc: 0.9190

Epoch 00004: val_acc did not improve from 0.92381
Epoch 5/50
19485/19485 [==============================] - 101s 5ms/step - loss

KeyboardInterrupt: 

In [18]:
print(K.get_value(aml_model.optimizer.lr))

0.0004


In [19]:
K.set_value(aml_model.optimizer.lr, 0.00015)

In [20]:
aml_model.fit(x_train_scaled,y_train_cat, batch_size = batch_size, epochs = 50, shuffle = True,class_weight=class_weight,
                        validation_data=(x_dev_scaled,y_dev_cat), callbacks=[ checkpointer])

Train on 19485 samples, validate on 840 samples
Epoch 1/50
19485/19485 [==============================] - 99s 5ms/step - loss: 1.4081 - acc: 0.9067 - val_loss: 0.4993 - val_acc: 0.9131

Epoch 00001: val_acc did not improve from 0.92500
Epoch 2/50
19485/19485 [==============================] - 100s 5ms/step - loss: 1.3034 - acc: 0.9116 - val_loss: 0.4773 - val_acc: 0.9190

Epoch 00002: val_acc did not improve from 0.92500
Epoch 3/50
19485/19485 [==============================] - 100s 5ms/step - loss: 1.2622 - acc: 0.9178 - val_loss: 0.5072 - val_acc: 0.9179

Epoch 00003: val_acc did not improve from 0.92500
Epoch 4/50
19485/19485 [==============================] - 101s 5ms/step - loss: 1.1192 - acc: 0.9213 - val_loss: 0.4793 - val_acc: 0.9202

Epoch 00004: val_acc did not improve from 0.92500
Epoch 5/50
19485/19485 [==============================] - 100s 5ms/step - loss: 1.2088 - acc: 0.9216 - val_loss: 0.4723 - val_acc: 0.9238

Epoch 00005: val_acc did not improve from 0.92500
Epoch 6/


KeyboardInterrupt

